#Summarizing Game Reviews with LLMs

## Importando os pacotes

In [ ]:
!pip install -q openai
!pip install -q langchain
!pip install -q tiktoken
!pip install -q requests
!pip install -q bs4
!pip install -q langdetect

In [ ]:
import openai
import tiktoken
import requests
import re
import pandas as pd
import numpy as np
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from langchain.docstore.document import Document
from bs4 import BeautifulSoup
from langdetect import detect

## Extraindo os dados do Metacritic

In [ ]:
# verifica se a próxima página de reviews existe
def nextPage(soup):
  if(soup.find(rel="next") == None):
    return False

  return True

In [ ]:
def web_scraping(link):
  v = link
  v = v[32:]
  div = v.find('/')
  platform = v[:div]
  v = v[div+1:]
  div = v.find('/')
  game = v[:div]
  v = v[div+1:]
  div = v.find('/')
  reviewer = v[:]

  print(f'Web Scraping: {game} of {platform} ({reviewer})')
  review_dict = {'name':[], 'date':[], 'rating':[], 'review':[]}
  page = 0

  url = link + '?page='+str(page)
  user_agent = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
  response  = requests.get(url, headers = user_agent)
  soup = BeautifulSoup(response.text, 'html.parser')

  while((reviewer == 'user-reviews' and (nextPage(soup) or page == 0)) or (reviewer == 'critic-reviews' and page == 0)):
    # print(f'Page {page}')

    for review in soup.find_all('div', class_='review_content'):
      if review == None:
        break

      if(reviewer == 'user-reviews'):
        if review.find('span', class_='blurb blurb_expanded'):
          rv = review.find('span', class_='blurb blurb_expanded').text
        else:
          if(review.find('div', class_='review_body').find('span') is None):
            rv = ''
          else:
            rv = review.find('div', class_='review_body').find('span').text
      else:
        if(review.find('div', class_='review_body') is None):
          rv = ''
        else:
          rv = review.find('div', class_='review_body').text

      try:
        language = detect(rv)
      except:
        continue

      if(language == 'en'):
        try:
          if(reviewer == 'user-reviews'):
            name = review.find('div', class_='name').find('a').text
            rating = int(review.find('div', class_='review_grade').find_all('div')[0].text)
          else:
            name = review.find('div', class_='source').find('a').text
            rating = int(review.find('div', class_='review_grade').find_all('div')[0].text)
            rating = round(rating/10)

          date = review.find('div', class_='date').text

        except:
          continue

        review_dict['name'].append(name)
        review_dict['date'].append(date)
        review_dict['rating'].append(rating)
        review_dict['review'].append(rv)

    page += 1
    url = link+'?page='+str(page)
    user_agent = {'User-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}
    response  = requests.get(url, headers = user_agent)
    soup = BeautifulSoup(response.text, 'html.parser')

  data = pd.DataFrame(review_dict)
  data['review'] = [" ".join(r.split()) for r in data['review']]
  return data

In [ ]:
link = "https://www.metacritic.com/game/playstation-3/the-last-of-us/user-reviews"
data = web_scraping(link)
data

Web Scraping: the-last-of-us of playstation-3 (user-reviews)


,name,date,rating,review
0,brad3ngunn,"Jun 14, 2013",10,I don't know what I can really say about this ...
1,Badonkadonk,"Jun 15, 2013",10,"I wouldn't call this game perfect, but it's pr..."
2,jakenelson,"Jun 18, 2013",10,I don't think I have ever played a game like t...
3,Qishere,"Jun 14, 2013",10,"You should realize right from the beginning, y..."
4,journeyfreak787,"Jun 14, 2013",10,This game truly is phenomenal. Naughty Dog pro...
...,...,...,...,...
2335,Aleklar,"May 18, 2023",10,This game is perfect This game is perfect This...
2336,OMGitsTHEMEMI,"May 18, 2023",10,This game is just perfection from the amazingl...
2337,AlesiLeite,"May 30, 2023",9,A game that gives you the feeling of immersing...
2338,Diven,"May 21, 2023",10,"Nice game really nice to play , waiting for pa..."


## Técnicas de Particionamento

**Simple Splitting**

In [ ]:
# Equal Size Partitioning
def simple_splitting(T,P=2000):
  # Data: Text T, Partition size P
  # Return: Partitions of text T

  tokens = re.split(' ', T)
  partitions = [' '.join(tokens[i:i+P]) for i in range(0,len(tokens),P)]

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. Five word sentences are fine."
simple_splitting(text, 4)

['This sentence has five',
 'words. Here are five',
 'more words. Five word',
 'sentences are fine.']

**Sliding Window**

In [ ]:
# Sliding Window Partitioning
def sliding_window_partitioning(T,w=2000,s=1800):
  # Data: Text T, Window size w, Step size s
  # Result: Partitions of text T using sliding windows

  tokens = re.split(' ', T)
  partitions = [' '.join(tokens[i:i+w]) for i in range(0,len(tokens),s)]

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. Five word sentences are fine."
sliding_window_partitioning(text, 4, 3)

['This sentence has five',
 'five words. Here are',
 'are five more words.',
 'words. Five word sentences',
 'sentences are fine.']

**Natural Boundary Splitting**

In [ ]:
def sentence_boundary(t):
  if re.match(r"\w*[.!?]", t):
    return True
  return False

In [ ]:
def review_boundary(t):
  if re.match(r"\n\n", t):
    return True
  return False

In [ ]:
# Boundary-based Partitioning
def boundary_based_partitioning(T,m=2000,B=review_boundary):
  # Data: Text T, Maximum partition size m, Boundary function B(t)
  # Result: List of partitions P

  tokens = re.split(' ', T)
  partitions = []
  p = []

  for t in tokens:
    if re.match(r"\w+[.!?]*", t):
      p.append(t)

    if(B(t) or len(p) == m):
      partitions.append(' '.join(p))
      p = []

  if(p != []):
    partitions.append(' '.join(p))

  return partitions

In [ ]:
# Multiple Boundaries Partitioning
def multiple_boundaries_partitioning(T,m=2000,B=review_boundary):
  # Data: Text T, Maximum partition size m, Boundary function B(t)
  # Result: List of partitions P

  tokens = re.split(' ', T)
  partitions = []
  p = []

  for t in tokens:
    if re.match(r"\w+[.!?]*", t):
      p.append(t)

    if(B(t)):
      if(len(p) >= m or len(p)+1 > m):
        partitions.append(' '.join(p))
        p = []

  if(p != []):
    partitions.append(' '.join(p))

  return partitions

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
boundary_based_partitioning(text, 4, sentence_boundary)

['This sentence has five',
 'words.',
 'Here are five more',
 'words.',
 'Five word sentences are',
 'fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
boundary_based_partitioning(text, 4, review_boundary)

['This sentence has five',
 'words. Here are five',
 'more words.',
 'Five word sentences are',
 'fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
multiple_boundaries_partitioning(text, 4, sentence_boundary)

['This sentence has five words.',
 'Here are five more words.',
 'Five word sentences are fine.']

In [ ]:
text = "This sentence has five words. Here are five more words. \n\n Five word sentences are fine."
multiple_boundaries_partitioning(text, 4, review_boundary)

['This sentence has five words. Here are five more words.',
 'Five word sentences are fine.']

## Técnicas de Amostragem

**Uniform Sampling**

In [ ]:
# Uniform Sampling
def uniform_sampling(C,n):
  # Data: Collection C, Sample size n
  # Result: Sample S

  return C.sample(n=n)

In [ ]:
uniform_sampling(data,10)

,name,date,rating,review
1353,bkshankz,"Oct 4, 2013",6,I got really bored by this game quickly. I thi...
204,whiterussian,"May 29, 2015",5,The game gets a lot of praise for it's story. ...
1517,Tromble,"Jun 29, 2021",9,The Last of Us is a fantastic game and a techn...
1673,MrZombie720,"Jun 28, 2013",8,This is a good game not a fantastic game like ...
12,Lethyss,"Jun 18, 2013",10,This is hands down the best game I've ever pla...
184,monkeyd100,"Jun 17, 2013",10,"This game is simply, Amazing! The story tellin..."
1600,DEFY,"Jun 22, 2013",10,Just beat the game on normal and it took me ab...
1728,RedDead94,"Oct 31, 2013",10,This is truly one of the best game of this gen...
837,SeraphCommander,"Jul 6, 2013",10,There is not much to say about the game apart ...
1542,Invader-Nik,"Jan 16, 2014",9,"This game is great. It's enjoyable, has a fant..."


**Stratified Sampling with Known Labels**

In [ ]:
# Stratified Sampling with Known Class Labels (Ratings)
def stratified_sampling_known_class_labels_ratings(C,n):
  # Data: Collection C with class labels L, Sample size n,
  # Result: Sample S

  S = pd.DataFrame(columns=['name','date','rating','review'])
  for L in range(0,11):
    cL = data[data['rating'] == L]
    nL = round(n*len(cL)/len(C))
    sL = cL.sample(n=nL)
    S = pd.concat([S,sL])

  return S

In [ ]:
stratified_sampling_known_class_labels_ratings(data,10)

,name,date,rating,review
1608,Psychosys,"Jun 30, 2014",8,"Great game, but nothing amazing. Story is the ..."
408,DeGonjo,"Jun 18, 2013",9,A must have if you own a PS3. 9.5/10 One of th...
394,deuranaer,"Jun 26, 2020",10,The Last of Us The best game ever released on ...
186,jackn,"Jun 17, 2013",10,"Well, The Last Of Us is like no other game I h..."
623,Gsomtin,"Jun 14, 2013",10,"Opinions are like butt holes, everyone has one..."
388,exalte,"Jun 25, 2020",10,"Best game ever, level design, animationts, lig..."
1460,Arfon,"Dec 4, 2020",10,This game is amazing. Perfect mix between a dr...
92,Sohail,"Aug 16, 2013",10,Naughty dog has done it again.A great game to ...
700,Matrosen,"Sep 1, 2013",10,The Last of Us is a must-play title for any PS...


In [ ]:
# Stratified Sampling with Known Class Labels (Groups of Ratings - Positive 8-10, Mixed 5-7 and Negative 0-4)
def stratified_sampling_known_class_labels_groups(C,n):
  # Data: Collection C with class labels L, Sample size n,
  # Result: Sample S

  S = pd.DataFrame(columns=['name','date','rating','review'])

  # Negative
  cL = data[data['rating'] <= 4]
  nL = round(n*len(cL)/len(C))
  sL = cL.sample(n=nL)
  S = pd.concat([S,sL])

  # Mixed
  cL = data[(data['rating'] >= 5) & (data['rating'] <= 7)]
  nL = round(n*len(cL)/len(C))
  sL = cL.sample(n=nL)
  S = pd.concat([S,sL])

  # Negative
  cL = data[data['rating'] >= 8]
  nL = round(n*len(cL)/len(C))
  sL = cL.sample(n=nL)
  S = pd.concat([S,sL])

  return S

In [ ]:
stratified_sampling_known_class_labels_groups(data,10)

,name,date,rating,review
2180,jordanlearmonth,"Jul 25, 2020",5,"The thing about videogames is, imo, they shoul..."
1709,TheBlackHeart,"Sep 6, 2013",10,The apocalypse is coming.. If I don't give a p...
1217,duffmanth,"Jun 20, 2013",9,I'm about 75% through this game right now and ...
667,MoultonHawk,"Jun 18, 2013",10,They made a mistake I think this game should h...
1327,chibistevo,"Jul 7, 2013",8,"Universal praise aside, The Last of Us is clea..."
2300,GAME-OVERcv,"Nov 1, 2022",10,perfect with pritty much everying but the best...
437,holdfasthope87,"Jun 19, 2013",10,From someone who has been a hardcore gamer for...
1739,handley35,"Dec 16, 2013",10,One word can only describe this game: AMAZING....
873,coven,"Aug 3, 2013",10,After seeing some incredible scores here on me...
2012,easternwind,"Jun 24, 2020",10,"Story,musics,gameplay everything is perfect. e..."


## Técnicas de Processamento

**Map Reduce**

In [ ]:
# Map-Reduce
def map_reduce(D,llm,prompt):
  # Data: Documents D, LLM, Map Prompt, Combine Prompt
  # Return: Summary

  summary = []

  # MAP
  for d in D:
    docs = [Document(page_content=t) for t in [d]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    sum = chain.run(docs)
    sum = " ".join(sum.split())
    summary.append(sum)

  # REDUCE
  if(len(summary) == 1):
    return summary[0]
  else:
    docs = [Document(page_content=t) for t in summary]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    final_summary = chain.run(docs)
    final_summary = " ".join(final_summary.split())
    return final_summary

**Elimination Processing**

In [ ]:
# Elimination Processing
def elimination_processing(D,llm,prompt, n=2):
  # Data: Documents D, LLM, Map Prompt, Combine Prompt, Eliminate n by n
  # Return: Summary

  summary = []

  # MAP
  for d in D:
    docs = [Document(page_content=t) for t in [d]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    sum = chain.run(docs)
    sum = " ".join(sum.split())
    summary.append(sum)

  # REDUCE
  while(len(summary)>1):
    docs = [Document(page_content=t) for t in summary[0:n]]
    del summary[0:n]

    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    sum = chain.run(docs)
    sum = " ".join(sum.split())

    summary.append(sum)

  final_summary = summary[0]
  return final_summary

**Sliding Window Processing**

In [ ]:
# Sliding Window Processing
def sliding_window_processing(D,llm,prompt, w=2, s=1):
  # Data: Documents D, LLM, Map Prompt, Combine Prompt, Window size w (eliminate w by w), Sliding size (repeat last w-s)
  # Return: Summary

  summary = []
  summary_aux = []

  # MAP
  for d in D:
    docs = [Document(page_content=t) for t in [d]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    sum = chain.run(docs)
    sum = " ".join(sum.split())
    summary.append(sum)

  # REDUCE
  while(len(summary)>1):
    docs = [Document(page_content=t) for t in summary[0:w]]
    del summary[0:s]

    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    sum = chain.run(docs)
    sum = " ".join(sum.split())

    summary_aux.append(sum)

    if(len(summary) < w):
      summary = summary_aux
      summary_aux = []

      if(len(summary) == 1):
        return summary[0]

  final_summary = summary[0]
  return final_summary

**Refine**

In [ ]:
# Refine
def refine(D,llm,prompt):
  # Data: Documents D, LLM, Refine Prompt
  # Return: Summary

  summary = ''

  # REFINE
  for d in D:
    docs = [Document(page_content=t) for t in [summary, d]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    summary = chain.run(docs)
    summary = " ".join(summary.split())

  return summary

**Weighted Refine**

In [ ]:
# Weighted Refine
def weighted_refine(D,llm,prompt):
  # Data: Documents D, LLM, Refine Prompt
  # Return: Summary

  summary = ''
  weight = 0

  # REFINE
  for d in D:
    Doc1 = f"Weight for current document: {weight}\n{summary}"
    Doc2 = f"Weight for current document: {1}\n{d}"
    docs = [Document(page_content=t) for t in [Doc1,Doc2]]
    chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)
    summary = chain.run(docs)
    summary = " ".join(summary.split())
    weight += 1

  return summary

In [ ]:
llm = OpenAI(
    model_name='text-davinci-003',
    openai_api_key=OPENAI_API_KEY,
    max_tokens=128)

prompt_template = """Following the reviews bellow, write a small summary:

{text}

SUMMARY OF THE REVIEWS:"""

PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

R = stratified_sampling_known_class_labels_ratings(data,10)
reviews = ''
for r in R['review']:
  reviews += r + ' \n\n '

texts = multiple_boundaries_partitioning(reviews, 600)

i = 1
for t in texts:
    print("Document " + str(i))
    print(t)
    i+=1

Document 1
First off the games all about a story, the menu opens up and it starts with a fart in the wind blowing and and an unlocked window. From what I take from it the games got wierd angled storyline, its about Child exploitation subliminally or even if theres no sex. I don't want to be a 40 year old with a 14 year old girl. the game is.18 and.over you're not.even allowed to.play it if your her age group. the subliminal aside i.cant follow the storyline at all. even stoned. I sold thjs game and the only.people that wanted it looked like incestual fathers and sex offenders who want to.see w 14 year old. after all the remakes and deprived games.sony made.i dont get the story line or hype. the actual game.is.good its got.a bit of everything snd the controls are not clunky. it's still.missing.controls though like.typical japanese graphics.are.best I ever seen in real.time bht i skipped through them and what I amnleft with is a typical game on rails. the ai doesnt change the replay valu

In [ ]:
map_reduce(texts, llm, PROMPT)

"The Last of Us is a critically acclaimed game from Naughty Dog that is praised for its immersive story, gripping gameplay, and realistic characters. It follows the journey of Joel as he struggles to protect Ellie in a dangerous post-apocalyptic world. Players are moved by the characters' struggles and have hailed it as one of the best video games ever, with its story being compared to the best movies and TV series. It is an unforgettable experience that is recommended to everyone."

In [ ]:
elimination_processing(texts, llm, PROMPT)

'The Last of Us by Naughty Dog is a critically acclaimed game that offers an intense and dramatic story, fantastic gameplay, and great voice acting, environment, characters, atmosphere and music. Players take the role of Joel in a post-apocalyptic world to deliver Ellie to a rebel group. Critics have praised the game for its story, gameplay, and characters, with many considering it one of the best games ever made.'

In [ ]:
sliding_window_processing(texts, llm, PROMPT)

'The Last of Us is a widely praised video game from Naughty Dog. It stands out from other survival games due to its intense story, engaging characters, and stunning visuals. Players take on the role of Joel and witness the growth of his relationship with Ellie. It has become a benchmark for the genre and is a must-play for any gamer.'

In [ ]:
refine(texts, llm, PROMPT)

'The Last of Us is a critically acclaimed game praised for its realistic graphics, engaging story, and characters players grow to love. It has been described as one of the best experiences on the PlayStation 3 due to its taut suspense and believable world. It is recommended for those looking for a deep and well-crafted game and is considered one of the greatest games ever made.'

In [ ]:
weighted_refine(texts, llm, PROMPT)

'The Last of Us is a highly praised video game released for the Playstation 3. Players have praised it for its realistic characters, tight game play, riveting story, emotionally charged moments, and its ability to make players think of what they would do in a similar situation. It is widely considered a masterpiece and many consider it to be the best PS3 game ever released. Despite frame-rate issues, it is still highly recommended. The story, characters, dialogue, and gameplay have all been praised and it is recommended as one of the top 5 best games ever made.'

## Resumindo as avaliações usando LLMs

In [ ]:
def summarize(R, S=stratified_sampling_known_class_labels_ratings, P=multiple_boundaries_partitioning, C=map_reduce):
  # Data: Reviews R (DataFrame), Sampling Function S, Partitioning Function P, Chain Type C
  # Result: Summarization of Reviews R

  LLM = OpenAI(
    model_name='text-davinci-003',
    openai_api_key='OPENAI_API_KEY',
    max_tokens=128)

  prompt_template = """Following the reviews bellow, write a small summary:

  {text}

  SUMMARY OF THE REVIEWS:"""

  PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])

  if(len(R) > 50):
    R = S(R,50)

  reviews = ''
  for r in R['review']:
    reviews += r + ' \n\n '

  TEXTS = P(reviews)

  return C(TEXTS, LLM, PROMPT)

In [ ]:
summarize(data,uniform_sampling,simple_splitting,map_reduce)

'The Last of Us is a critically acclaimed video game praised for its emotional narrative, intense gameplay, and impressive visuals. It follows two characters, Joel and Ellie, in a post-apocalyptic world and features unique character development and environment-based combat. It is praised for its strong soundtrack and voice acting, and many consider it to be the best game of all time. However, some have found it to be too slow-paced and overrated. It is a difficult game with occasional frame rate drops, but is still an enjoyable experience that should not be missed.'

In [ ]:
summarize(data,uniform_sampling,simple_splitting,elimination_processing)

'The Last of Us is a highly praised video game critically acclaimed for its engaging combat, stunning visuals, and immersive multiplayer mode. It is praised for its emotionally charged story, believable characters, and engaging gameplay, as well as its immersive world, intense characters, and realistic graphics and sound. Despite some criticism regarding AI, button mashing, and linear story, the game is seen as a masterpiece for its beautiful graphics, intense combat, and emotionally gripping narrative.'

In [ ]:
summarize(data,uniform_sampling,simple_splitting,sliding_window_processing)

'The Last of Us has been praised for its emotional story, engaging gameplay, and realistic graphics, but there are some issues such as the hand to hand combat, lack of music, and immersion-breaking elements. Despite its flaws, many consider it one of the best games of all time and a must-play for all gamers.'

In [ ]:
summarize(data,uniform_sampling,simple_splitting,refine)

'The Last of Us for the PS3 is widely acclaimed as one of the best games of its generation, praised for its emotional story, engaging characters, and intense gameplay. The graphics, sound, voice acting, and multiplayer mode are all praised, though there are some flaws such as a lack of replayability and some bugs. Players have a wide range of opinions, with some finding the experience to be gripping and emotionally resonant, while others find the story cliche or the gameplay tedious. However, most agree that it is a masterpiece and well worth playing.'

In [ ]:
summarize(data,uniform_sampling,simple_splitting,weighted_refine)

'The Last of Us is a critically acclaimed survival horror game that has mixed reviews but is generally praised for its storytelling, visuals, and unique gameplay. It is seen as one of the best games of all time by some, and is highly recommended by most. It has an engrossing story, compelling characters, great gameplay and atmosphere, and is considered a masterpiece by many. However, some criticize its AI and ending.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,map_reduce)

"The Last of Us is a critically acclaimed masterpiece from Naughty Dog that is praised for its emotionally charged story, thrilling and responsive gameplay, and stunning graphics. Players take the role of Joel and Ellie as they traverse a broken America, and the realistic violence adds to the game's dark and desperate tone. Multiplayer is a little bare, but the single player campaign provides an intense and emotionally draining experience. Despite some minor flaws, such as AI issues and some bugs, the overall package is well worth the play."

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,elimination_processing)

'The Last of Us is a critically acclaimed masterpiece of a game that has been praised for its visuals, gripping story, believable characters, and intense gameplay. It is a unique experience that successfully blurs the lines between movies and games and is sure to be remembered for years to come.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,sliding_window_processing)

'The Last of Us is a critically acclaimed game that is praised for its outstanding graphics, emotionally engaging story, deep characters, intense gameplay, and replay value. It is an unforgettable experience that is worth the purchase despite its flaws, and is enjoyable for both single-player and multiplayer. It requires skill and strategy and is a must-play game that offers a rewarding and immersive experience. It has realistic brutality, but is not a shooting gallery and requires caution. It leaves the possibility for a sequel, though its ending may be unsatisfactory.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,refine)

'The Last of Us is an acclaimed title praised for its visuals, soundtrack, and atmosphere. It features genre-defining mechanics, an emotionally engaging story and a unique relationship between Joel and Ellie. Despite occasional AI bugs, it is considered one of the best games of the generation and is highly recommended. Critics have praised the game for its story, atmosphere, and audio, as well as its unique action and emotion. The graphics and level design are praised, although there may be some issues with the gunplay and repetitive nature of the game. Ultimately, The Last of Us is an unforgettable experience that will leave a lasting impression.'

In [ ]:
summarize(data,uniform_sampling,sliding_window_partitioning,weighted_refine)

'The Last of Us is an acclaimed action game developed by Naughty Dog that has been praised for its beautiful graphics, story, engaging characters, and challenging gameplay. It is widely considered one of the best games ever made and is praised for its emotional connection with the characters, its tension-filled gameplay, and its stunning visuals. Though some argue that it has flaws such as its zombie-like enemies, it is still an overall must-play experience.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,map_reduce)

'The Last of Us is an acclaimed post-apocalyptic video game from Naughty Dog featuring great graphics, sound, AI, and an immersive story. It has been praised for its cinematic story-telling, emotional characters, intense atmosphere, and intense horror elements. It has been rated as one of the best games of 2013 and has a strong replay value. It has been criticized for its one-dimensional characters and repetitive gameplay, but is still considered one of the best games ever made.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,elimination_processing)

'The Last of Us is an acclaimed title from Naughty Dog that has been praised for its captivating story, excellent voice acting, stunning visuals, and immersive sound design. It has won numerous awards and is widely considered one of the best PlayStation games of all time. It features realistic and lovable characters, and a gripping and believable story, however the gameplay and friendly AI could use some innovation. Despite some minor flaws, it is an unforgettable experience that will leave players feeling powerful emotions and never lose hope.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,sliding_window_processing)

'The Last of Us is an acclaimed game from Naughty Dog, praised for its amazing story, visuals, soundtrack, characters and gameplay. It combines adventure, survival, action, stealth and exploration, with great graphics and an emotional story. Despite some feeling it is overrated and not revolutionary, it is still considered one of the greatest games of all time and is a must-play for gamers.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,refine)

'The Last of Us is a critically acclaimed post-apocalyptic game that features an excellent story, realistic combat, stunning visuals, and an emotional soundtrack. Players join Joel and Ellie on an emotional journey as they navigate a dangerous post-apocalyptic world. Although the plot may be a bit underwhelming, the characters and story are captivating and engaging. The game is praised for its immersive gameplay, sound design, and deep themes. Despite a strange ending, The Last of Us is a must-play for fans of the genre.'

In [ ]:
summarize(data,uniform_sampling,multiple_boundaries_partitioning,weighted_refine)

"The Last of Us is an acclaimed post-apocalyptic game by Naughty Dog that features stunning visuals, an intense and realistic combat system, an emotionally captivating story, and an immersive and captivating gaming experience. Critics have argued that the game doesn't quite live up to the immense hype it has received, but many agree it is one of the best games of all time. It incorporates a great story telling and top notch gameplay with a light crafting system. It also has a unique multiplayer experience with a faction system that adds to the replayability. The Last of Us delivers an unforgettable experience with an intense, emotional, and engaging story that"

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,simple_splitting,map_reduce)

'The Last of Us is a critically acclaimed video game from Naughty Dog that has won the hearts of many players. It has stunning graphics, immersive story-telling, intense combat, crafting elements, and an awesome cast of characters. Despite some flaws such as clunky AI and repetitive gameplay, the positives of the game far outweigh the negatives. It is a must-have for any PS3 owner and is highly recommended for gamers of all levels.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,simple_splitting,elimination_processing)

'The Last of Us is a critically acclaimed game that is considered one of the best of all time. It features stunning visuals, a deep and emotionally gripping story, great controls, and intense multiplayer. It has some minor bugs and glitches, but nothing that detracts from the experience. It is a must-own game for any Playstation 3 owner.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,simple_splitting,sliding_window_processing)

'The Last of Us is widely regarded as one of the best games of this generation, praised for its stunning graphics, narrative, performances, and gameplay. Despite some flaws such as unpredictable AI and too-easy supply acquisition, the overall experience is highly immersive and emotionally powerful, making it a must-play for any fan of video games.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,simple_splitting,refine)

'The Last of Us is universally acclaimed for its incredibly immersive story, characters, graphics, sound, and animations. It features intense combat and offers a unique post-apocalyptic experience that draws players in and never lets go. It is praised for its depth and emotional connection to the characters, with some claiming it to be one of the best games ever made. Despite some AI and difficulty issues, it is still a highly enjoyable game that stays with players long after the credits roll.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,simple_splitting,weighted_refine)

'The Last Of Us is highly praised for its stunning visuals, emotionally moving story, unpredictable plot, and combat system. The multiplayer was seen as comparatively poor to the single-player. It is widely considered to be the best game of the generation, and highly recommended for gamers of all types. It has earned many awards and is praised for its immersive story, characters, and atmosphere. It features unique gameplay with limited resources and no health regeneration, and has some graphical glitches and AI issues. Overall, it is an unforgettable experience that is praised for its story and gameplay.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,sliding_window_partitioning,map_reduce)

'The Last of Us is a critically acclaimed masterpiece from Naughty Dog that features stunning graphics, realistic sound effects, great voice acting, and an emotionally charged storyline. It immerses players into a post apocalyptic world filled with danger and suspense, requiring strategic thinking and improvisation to survive. With its replayability, script, cinematography, and horror elements, it has revived interest in the PS3 and consoles in general. It has bugs and the AI is not perfect, but it is a must-play game for mature gamers.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,sliding_window_partitioning,elimination_processing)

'The Last of Us is a critically acclaimed post-apocalyptic game from Naughty Dog that features an emotionally charged story, captivating gameplay, stunning visuals, and intense atmosphere. Its crafting system and multiplayer mode emphasize teamwork and survival, though there can be inconsistencies with the AI and some repetitiveness. Despite this, it is a great game and a must-play experience for any mature gamer.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,sliding_window_partitioning,sliding_window_processing)

'The Last of Us is a highly praised game that has earned widespread acclaim for its emotionally engaging story, attractive visuals, and intense combat. Despite some criticisms regarding AI and puzzles, the game is generally seen as a masterpiece and has become a favorite amongst gamers of all ages and backgrounds.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,sliding_window_partitioning,refine)

'The Last of Us is a critically acclaimed game praised for its stunning visuals, atmospheric music, and emotional story. It follows Joel and Ellie as they travel through a post-apocalyptic world, and although there may be some repetitive elements and difficult AI, the game is immersive and engrossing. It has been lauded for its beautiful visuals, emotional story, and gripping gameplay.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,sliding_window_partitioning,weighted_refine)

'The Last of Us from Naughty Dog is a critically acclaimed game that has earned praise for its engaging story, visuals, characters, and gameplay. Players appreciate the game despite some issues with linearity, repetitive combat, and AI. It has been described as an emotionally charged experience that immerses players in a post-apocalyptic world and encourages them to question their morality. Many reviewers highly recommend it to anyone looking for a game that will leave a lasting impression. It has been praised for its production values, immersive world, and emotional narrative. It is widely considered to be one of the best games of all time.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,multiple_boundaries_partitioning,map_reduce)

'The Last of Us is a genre-defining third-person shooter game with stealth elements from Naughty Dog that has received universal praise for its visuals, characters, story, sound design, and gameplay. It follows Joel and Ellie, who must survive in a post-apocalyptic world after a virus. The reviews praised the game for its immersive and emotional story, realistic gunplay and stealth, crafting, and a fun multiplayer mode. The game has become a testament to the debate over whether video games can be considered art, with its attention to detail, impressive visuals, and emotionally engaging performances from its actors. The Last of Us'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,multiple_boundaries_partitioning,elimination_processing)

'The Last of Us is generally praised for its immersive presentation, captivating story, and tactical gameplay. While there are some criticisms of its repetitive game-play, poor AI, and unsatisfying ending, it is still considered a great game overall and is compared to some of the greatest games of all time.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,multiple_boundaries_partitioning,sliding_window_processing)

'The Last of Us is a highly praised game, seen by many as a masterpiece with an emotional story, immersive gameplay, and near-realistic graphics. It is a benchmark for single-player campaigns, with its unique multiplayer mode being both challenging and fun. Despite some criticism of its repetitive gameplay and cliched plot, it is almost universally praised and should not be missed.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,multiple_boundaries_partitioning,refine)

"The Last of Us is a PS3 exclusive game highly praised for its cinematic storytelling, graphics, soundtrack, upgrade system, crafting and journal-using mechanics, and character development. There are criticisms of the game's repetitive gameplay and dumb AI, but it is still recommended for its emotional journey and beauty. It is a unique masterpiece that has made many players feel with the characters and kept them thrilled throughout the game. It is praised for its excellent audio, voice acting, and graphics, and is considered one of the greatest games of all time."

In [ ]:
summarize(data,stratified_sampling_known_class_labels_ratings,multiple_boundaries_partitioning,weighted_refine)

"The Last of Us is an acclaimed post-apocalyptic game praised for its story, characters, atmosphere, and graphics. It is unanimously rated as a must-play for PlayStation 3 players and features a mix of stealth and action gameplay, as well as an adventure element and multiplayer mode. It is considered one of the best games ever made for its emotional depth and compelling characters. Players have praised the game's visuals, plot-line, character development, and voice acting, while also noting the tough and realistic fighting. The Last of Us is deemed a masterpiece for its story, which features layers of messages and takeaways, and its characters,"

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,simple_splitting,map_reduce)

'The Last of Us is widely praised as one of the best games of the PS3 era. It has stunning graphics, an emotionally gripping story, and intense gameplay. It features a crafting system and combat that takes some time to master, and the acting and character development are praised. There are some criticisms about its difficulty curve and lack of urgency, but overall, it is considered a classic and unforgettable experience.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,simple_splitting,elimination_processing)

'The Last of Us is a highly praised game that has been lauded for its emotionally engaging narrative, stunning visuals, and tense and immersive gameplay. It has been considered a masterpiece of storytelling, visuals, and gameplay and is recommended for anyone looking for a great story and an immersive gaming experience. Although some have found the AI companions to be lacking and the story to be cliche, the game is still highly recommended and the multiplayer is enjoyable.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,simple_splitting,sliding_window_processing)

'The Last of Us from Naughty Dog is a highly praised post-apocalyptic and survival horror game that has been lauded for its cinematic storytelling, lifelike animations, creative gameplay, and replay value. It has been hailed as one of the best games of the seventh generation and the best game of 2013. Although some minor grievances such as AI interaction and lack of content have been noted, the game is still seen as perfect and is given a full 10/10 rating.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,simple_splitting,refine)

'The Last Of Us is hailed as a masterpiece due to its incredible story, graphics, and voice acting. It has won over 200 awards and is considered one of the greatest games of all time. Players are pulled into a post-apocalyptic world full of danger, and must use limited ammo to face intense and challenging scenarios. Despite some minor glitches and lack of character development, The Last Of Us is an all-time classic and a testament to the art form of video games.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,simple_splitting,weighted_refine)

'The Last of Us is widely acclaimed as one of the best video games of all time, praised for its emotionally gripping story, realistic graphics, and engaging atmosphere. While it has been criticized for its AI issues and linearity, many people still consider it to be an essential piece of interactive entertainment. It has earned near unanimous praise for its masterful narrative, beautiful graphics, and engaging gameplay. It is a unique experience that can be enjoyed by fans of all types of games.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,sliding_window_partitioning,map_reduce)

'The Last of Us is a critically acclaimed post-apocalyptic game that has become a beloved classic. It features stunning visuals, immersive gameplay, and an emotional story with characters that draw players in. The game has earned an impressive number of perfect scores and multiple awards, making it a must-play for any fan of the genre or gaming in general. It is praised for its character writing, setting, and relationship between Joel and Ellie, as well as its tactical and stealth-based gameplay and online mode. Despite its few criticisms, such as repetitive elements and predictable plot, it is highly recommended and considered to be one of the best'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,sliding_window_partitioning,elimination_processing)

'The Last of Us is a critically acclaimed post-apocalyptic game with realistic gameplay, intense action sequences, a powerful story, and a great soundtrack. It is praised for its gripping story, beautiful visuals, immersive gameplay, and replayability, and is considered a must-play for any PlayStation 3 owners.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,sliding_window_partitioning,sliding_window_processing)

'The Last of Us is a critically acclaimed video game developed by Naughty Dog that has earned praise for its engaging story, gripping plot, realistic characters, stunning visuals, and intense tactical combat. Although there are some issues with its third-person shooter gameplay, quicktime events, and generic and forgettable moments, the game is still highly regarded and is seen as one of the greatest games ever. It is a must-play for any fan of the genre.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,sliding_window_partitioning,refine)

'The Last of Us is a critically acclaimed masterpiece from Naughty Dog that has been praised for its emotionally impactful story and characters, intense and realistic gameplay, award-winning voice acting, and gorgeous graphics. Fans of the game note some minor flaws such as a lack of weapon variety and some bugs, yet it is still considered one of the best video games ever created. With its heart-wrenching story and unforgettable gaming experience, The Last of Us is a must-play for any serious video game fan.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,sliding_window_partitioning,weighted_refine)

'The Last of Us is a critically acclaimed game that has won universal 10/10 scores and is highly praised for its powerful story, great characters, satisfying gameplay, stunning graphics, and immersive atmosphere. Despite some criticisms of its linearity and repetitive elements, many consider it a masterpiece. The reviewer personally gave it a 9.7/10 rating, noting that while there are some minor issues, it is still an amazing game and is highly recommended.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,multiple_boundaries_partitioning,map_reduce)

'The Last of Us is a highly acclaimed survival horror game for Playstation 3, developed by Naughty Dog and released in 2013. It received widespread praise for its compelling story, intense atmosphere, and engaging gameplay. Players have praised the visuals, sound, and crafting mechanics, but some have criticized it for its overly scripted combat scenes and lack of exploration. Despite some flaws, it is considered one of the best games of all time and has been deemed a masterpiece by many.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,multiple_boundaries_partitioning,elimination_processing)

'The Last of Us is widely acclaimed as a modern masterpiece, praised for its emotional storytelling, memorable characters, beautiful graphics, and strategic gameplay. It is one of the best games of its generation, with immersive gameplay, stunning visual and sound design, and a unique combination of stealth, shooting, and resource management. Despite some minor bugs and glitches, it has been hailed as one of the best games of all time.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,multiple_boundaries_partitioning,sliding_window_processing)

'The Last of Us is widely praised for its engaging story, visuals, voice acting, and immersive gameplay. While some reviewers have noted certain shortcomings, it is still considered an excellent game and a must-play for PlayStation owners. It features intense stealth/action gameplay, well-crafted characters, and a refined multiplayer mode.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,multiple_boundaries_partitioning,refine)

'The Last of Us is widely praised for its story, characters, graphics, and immersive atmosphere. It is often considered one of the best games of the past generation and is a must-have for those wanting to experience an emotionally charged, cinematic story-driven game. It has some flaws such as linearity and AI, but its overall experience is worth it.'

In [ ]:
summarize(data,stratified_sampling_known_class_labels_groups,multiple_boundaries_partitioning,weighted_refine)

"The Last of Us has been widely praised for its story, characters, graphics, sound, cinematics, and replay value. Although some criticize the repetitive and unimaginative gameplay, it has been acclaimed for its creative ideas and classic gaming elements. It is often considered one of the best games of the generation and is remembered for its emotional storyline and ending. Naughty Dog's new IP from Neil Druckmann is a brutalist survival masterpiece, with incredible graphics, a great story and balanced stealth gameplay. It is highly recommended, and some even believe it is the best game ever made."